In [56]:
import os
import math
import time 
import numpy as np
import tensorflow as tf
import pandas as pd
import matplotlib.pyplot as plt
from tensorflow.keras import layers, models
from tensorflow.keras.preprocessing.image import ImageDataGenerator

In [57]:
os.environ["CUDA_VISIBLE_DEVICES"] = "1"
label_classes = ['森森', '小白']
# label_classes = ['森森', '牛排', '巧巧', '小白', '花捲', '金毛', '巧虎', '超人', '吊吊', '黑胖', '乖狗', '橘子']

# num_of_class = 12
batch_size = 6

In [58]:
train_datagen = ImageDataGenerator(
    rescale=1./255,
    # shear_range=0.1,
    # zoom_range=0.1,
    # width_shift_range=0.1,
    # height_shift_range=0.1,
    # horizontal_flip=True,
    # vertical_flip=True,
    # validation_split=0.1
)

valid_datagen = ImageDataGenerator(
    rescale=1./255
)

train_generator = train_datagen.flow_from_directory(
    directory="data/",
    target_size=(224,224),
    color_mode="rgb",
    class_mode='categorical',
    classes=label_classes,
    batch_size=batch_size,
    shuffle=True,
)

valid_generator = train_datagen.flow_from_directory(
    directory="data/",
    target_size=(224,224),
    color_mode="rgb",
    class_mode='categorical',
    classes=label_classes,
    batch_size=batch_size,
    shuffle=True,
)

# for image_batch, label_batch in train_generator:
#     break
# print("shape of X_train:", image_batch.shape)
# print("shape of y_train:", label_batch.shape)
    

Found 36 images belonging to 2 classes.
Found 36 images belonging to 2 classes.


In [59]:
# for my_batch in train_generator:
#     images = my_batch[0]
#     labels = my_batch[1]
#     for i in range(len(labels)):
#         plt.imshow(images[i])
#         plt.colorbar()
#         plt.show()
#         print(labels[i])
#     break

In [60]:
base_model = tf.keras.applications.MobileNetV3Large(
    include_top = False,
    input_shape=(224,224,3),
    weights = 'imagenet',
    # classes = 2,
    # classifier_activation = "softmax"
)

base_model.trainable = False

model = models.Sequential([
    base_model,
    layers.Conv2D(128, 3, activation='relu'),
    layers.GlobalAveragePooling2D(),
    layers.Dense(2, activation='softmax')
])

model.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 MobilenetV3large (Functiona  (None, 7, 7, 960)        2996352   
 l)                                                              
                                                                 
 conv2d_6 (Conv2D)           (None, 5, 5, 128)         1106048   
                                                                 
 global_average_pooling2d_6   (None, 128)              0         
 (GlobalAveragePooling2D)                                        
                                                                 
 dense_6 (Dense)             (None, 2)                 258       
                                                                 
Total params: 4,102,658
Trainable params: 1,106,306
Non-trainable params: 2,996,352
_________________________________________________________________


In [61]:
# model.compile(
#     optimizer=tf.keras.optimizers.Adam,
#     loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
#     metrics=[tf.keras.metrics.SparseCategoricalAccuracy()],
# )

# my_callback = [tf.keras.callbacks.ModelCheckpoint(
#     filepath='model_save/model_{epoch:03d}'
# )]

model.compile(optimizer=tf.keras.optimizers.Adam(),
              loss=tf.keras.losses.BinaryCrossentropy(),
              metrics=['accuracy']
              )

history = model.fit(
    train_generator,
    epochs=10,
)

# acc = history.history['accuracy']
# epochs = range(1, len(acc) + 1)
# val_acc = history.history['val_accuracy']
# plt.plot(epochs, acc, 'b', label='Training acc')
# plt.plot(epochs, val_acc, 'r', label='Validation acc', linewidth=0.5)
# plt.title('Training and validation accuracy')
# plt.legend(loc='lower right')
# plt.grid()
# # 儲存acc學習曲線
# # plt.savefig('./acc.png')
# plt.show()

# # 畫出loss學習曲線
# loss = history.history['loss']
# val_loss = history.history['val_loss']
# plt.plot(epochs, loss, 'b', label='Training loss')
# plt.plot(epochs, val_loss, 'r', label='Validation loss', linewidth=0.5)
# plt.title('Training and validation loss')
# plt.legend(loc='upper right')
# plt.grid()
# # 儲存loss學習曲線
# # plt.savefig('loss.png')
# plt.show()

Epoch 1/10
6/6 [==============================] - 9s 932ms/step - loss: 0.7445 - accuracy: 0.5000
Epoch 2/10
6/6 [==============================] - 6s 976ms/step - loss: 0.8207 - accuracy: 0.5000
Epoch 3/10
6/6 [==============================] - 6s 971ms/step - loss: 0.5660 - accuracy: 0.7778
Epoch 4/10
6/6 [==============================] - 6s 988ms/step - loss: 0.5328 - accuracy: 0.7500
Epoch 5/10
6/6 [==============================] - 6s 960ms/step - loss: 0.4279 - accuracy: 1.0000
Epoch 6/10
6/6 [==============================] - 6s 995ms/step - loss: 0.3581 - accuracy: 1.0000
Epoch 7/10
6/6 [==============================] - 6s 1s/step - loss: 0.2734 - accuracy: 1.0000
Epoch 8/10
6/6 [==============================] - 6s 1s/step - loss: 0.2221 - accuracy: 1.0000
Epoch 9/10
6/6 [==============================] - 6s 1s/step - loss: 0.1736 - accuracy: 1.0000
Epoch 10/10
6/6 [==============================] - 6s 952ms/step - loss: 0.1247 - accuracy: 1.0000


In [62]:
pred_train = model.evaluate(train_generator, verbose=0)
pred_val = model.evaluate(valid_generator, verbose=0)

print("Train loss:", pred_train[0])
print("Train accuracy:", pred_train[1])
print("Validation loss:", pred_val[0])
print("Validation accuracy:", pred_val[1])

Train loss: 0.11486204713582993
Train accuracy: 1.0
Validation loss: 0.11486204713582993
Validation accuracy: 1.0
